In [26]:
#Library
import requests
import json
from datetime import datetime
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.precision", 12)
import warnings
warnings.filterwarnings('ignore')

In [27]:
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    payload = {'date': date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi,# + columns_to_str(columns),
                      params=payload,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [28]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.post(url=URL + endpointApi,# + columns_to_str(columns),
                      data=content,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

Conversion chaque valeur en le type approprié :

In [29]:
def convert_currency(value, df_convert_currency):
    value, currency_name = value.split(' ')
    rate = currencyRateToEUR(currency_name, df_currency_rate)
    
    new_value = float(value.replace(',', '.')) * rate
    return new_value

def convert_datetime(value):
    return datetime.strptime(value, '%Y-%m-%d').date()

def convert_float(value):
    return float(value.replace(',', '.'))

def convert_int(value):
    return int(value)

def convert_boolean(value):
    return json.loads(value)

def df_convert_type(df, df_currency_rate):
        for col in df.columns:
            convert_values = []
            for elt in df[col]:
                if elt is np.nan:
                    convert_values.append(np.nan)
                    continue

                elt_type = elt['type']
                elt_value = elt['value']
                if elt_type == 'currency_value':
                    elt_value = convert_currency(elt_value, df_currency_rate)
                elif elt_type == 'date':
                    elt_value = convert_datetime(elt_value)
                elif elt_type in ['double', 'percent']:
                    elt_value = convert_float(elt_value)
                elif elt_type in ['asset', 'int32']:
                    elt_value = convert_int(elt_value)
                elif elt_type == 'boolean':
                    elt_value = convert_boolean(elt_value)
                elif elt_type not in ['asset_type', 'string', 'asset_currency', 'date-time',
                                     'asset_sub_type', 'asset_status', 'asset_quote_type',
                                     'liquidity_algorithm', 'portfolio_lock_mode', 'portfolio_type']:
                    print(elt)
                convert_values.append(elt_value)
            df[col] = convert_values
        return df

Création de la table des taux en EUR : 
 
[Est-ce qu'on donne la posibilité de changer la currency dst en fonction de la currency du portfolio ?]

In [30]:
def getCurrencyRate():
    d = []
    currency_json = json.loads(get_data('currency'))
    for currency in currency_json:
        currency_id = currency.get('id')
        rate = get_data('currency/rate/{}/to/EUR'.format(currency_id))
        if len(rate) == 0:
            continue
        rate_value = json.loads(rate)['rate']['value']
        d.append([currency_id, convert_float(rate_value)])
    return pd.DataFrame(d, columns=['currency src', 'rate'])
    
df_currency_rate = getCurrencyRate()
df_currency_rate

,currency src,rate
0,USD,0.851208716377
1,EUR,1.000000000000
2,JPY,0.008066467694
3,BEF,0.024789352477
4,CYP,1.708601441376
...,...,...
23,LUF,0.024789352477
24,VAL,0.000516456899
25,SML,0.000516456899
26,EUr,0.010000000000


[TEST] Taux de convertion entre USD et EUR :

In [31]:
def currencyRateToEUR(cur_src, df_currency_rate):
    return df_currency_rate[df_currency_rate['currency src'] == cur_src]['rate'].values[0]
rate = currencyRateToEUR('USD', df_currency_rate)
rate


0.851208716377

[TEST] La database complète de tous les asset :

In [32]:
data = get_data('asset/')
df = pd.read_json(data)
df_ETF = df_convert_type(df, df_currency_rate)
df_ETF

,MARKET_PLACE_MIC,LAST_GROSS_VALUE,LAST_QUOTE_DATE,quote_value_type,MARKET_PLACE_CURRENCY,CATEGORY_CIC,CREATION_DATE,LIQUIDITY_ALGO,FIRST_QUOTE_DATE,LAST_QUOTE_SOURCE,REST_UNIQUE_NUMBER,VL_DELAY,SUB_CATEGORY_CIC,MARKET_CAP_CATEGORY,INDEXED_RATE,GRAPH_PAST_WEEK,GRAPH_LAST_YEAR,PORT_LINE_COUNT,IS_PUBLICLY_TRADED,MARKET_PLACE_CITY,IS_PORTFOLIO,DAILY_PRICE_COUNT,CITY_OR_ACRONYM_OR_NAME,IS_STRUCTURED_PRODUCT,MARKET_PLACE,LAST_QUOTES_UPDATE_DATE,SUB_SECTOR,MARKET_PLACE_DESC,GRAPH_PERIOD,INDUSTRY,CURRENCY,GRAPH_PAST_YEAR,asset_custom_classification_bloomberg,COUNTRY_CIC,GLOBAL_TYPE,COUNTRY,ASSET_OR_MARKET_PLACE_COUNTRY,LAST_CLOSE_VALUE_IN_CURR,LAST_CLOSE_VALUE,MODIFICATION_DATE,GICS_INDUSTRY_GROUP,PORT_CONTENT_COUNT,EMIR_CLEARING_ID_TYPE,SEDOL_CODE,IS_FUND_FED,STOCK_TYPE,COUNTRY_ISO_3,FINAL_UNDERLYING,UNDERLYING_OR_ASSET_IF_UNDERLYING_NOT_AVAILABLE,MARKET_PLACE_TELEKURS,GICS_SECTOR,GRAPH_LAST_6_MONTH,BLOOMBERG_CODE,asset_attr_init,IS_ACCESSIBILITY_PUBLIC,ASSET PRICE MODE,MARKET_PLACE_OR_ASSET_COUNTRY,MARKET_PLACE_COUNTRY,ASSET_DATABASE_ID,ASSET_STATUS,TYPE,CODE_CIC,GICS_SUB_INDUSTRY,QUOTE_TYPE,IS_EURONEXT,IS_COMPOSITE,FINAL_UNDERLYING_TYPE,IS_FUND_SAVE_SENS,GICS_INDUSTRY,SUB_TYPE,SECTOR,BLOOMBERG_ID,IS_DEPOSITARY_QUOTES,LABEL,PAY_ACCOUNT_ON_DEFAULT,IS_EUROLAND,MARKET_PLACE_TYPE,SUP_SECTOR,ISIN_CODE,asset_custom_classification_Tab. de bord GSM,FLOATING_VALUE,REST_OBJECT_ID,ASSET_MARKET_SUB_GROUP,ELIGIBLE_PEA_PME,ELIGIBLE_PEA,asset_fund_info_asset_class_focus,TRANSPARENCY_LIST,BOND_RISK_COMPUTATION_MODE,asset_fund_info_strategy,asset_fund_info_maximum_front_end_fee,asset_fund_info_bloomberg_ticker,asset_fund_info_is_feeded,asset_fund_info_authorized_for_retail_investors,BOND_CREDIT_RISK_COMPUTATION_MODE,asset_fund_info_fund_leverage,asset_fund_info_objective_long,asset_fund_info_is_euro_ucits,BOND_WAL_COMPUTATION_MODE,asset_fund_info_administrator,asset_fund_info_isin_code,BOND_WAM_COMPUTATION_MODE,asset_fund_info_minimum_investment_shares,asset_fund_info_depositary,asset_fund_info_manager_stated_fee,BOND_YTM_COMPUTATION_MODE,asset_fund_info_cut_off,asset_fund_info_maximum_back_end_fee,asset_fund_info_company,asset_fund_info_vl_date,IS_REAL_ESTATE_FUND,asset_fund_info_manager_stated_perf_fee,asset_fund_info_auditor,FUND_TYPE,asset_fund_info_fund_type,asset_fund_info_valorisation_frequency,asset_fund_info_creation_date,MIN_BUY_INITIAL_AMOUNT,asset_fund_info_pct_max_opcvm,asset_fund_info_initial_minimum_subscription,asset_fund_info_fund_open_pymt_shr,asset_fund_info_amf_classification,asset_fund_info_decimalisation,MIN_BUY_AMOUNT,asset_fund_info_amf_date,asset_fund_info_minimum_subscription,asset_fund_info_day_cut_off,asset_fund_info_vl_known,asset_fund_info_geographical_focus,asset_fund_info_subscription_committee,asset_fund_info_abide_with_the_four_criterias,asset_fund_info_reuters_code,REUTERS_CODE,asset_fund_info_subscription_frequency,asset_fund_info_redemption_frequency,VALOREN_CODE,asset_fund_info_valoren_code,asset_fund_info_manager,asset_fund_info_redemption_committee,asset_fund_info_advised_horizon,FUND_REF_IDX,asset_fund_info_comanager,HAS_TOBIN_TAX,CUSIP_CODE,LOCAL_CODE,asset_fund_info_rate_entry,asset_fund_info_management_style,asset_fund_info_rate_output,asset_fund_info_rate_entry_sdg,asset_fund_info_rate_output_sdg,MARKET_QUOTE_IDENTIFIER,asset_fund_info_manager_committee,PORT_ASSET_CONTENT_COUNT,INDEX_TYPE,QUOTE_PLACE,TELEKURS_QUOTE_PLACE_CODE,IS_SRD,PORT_IS_HISTORICAL,PORT_BASE_NAV,PORT_VL,REAL_TIME_PORT_CASH,PORT_FEED_TYPE,LOCK_MODE,PORT_CASH,PORT_NET_EX_CASH_EQUIVALENT,NB_COMPTE_ON_PORTFOLIO,PORT_TYPE,EAI_FRONTPORT_CASH_BY_CRCY,PRICER_MODE,PORT_CASH_AVAILABLE,PORT_STATUS,EAI_BACKPORT_CASH_BY_CRCY,EAI_PORTFOLIO_FEED_NAME,PORT_NET_EX_CASH,PORT_FEED_CODE,PORT_NEED_RECONCILIATION,PORT_CLEAN_MODE,PORT_CURRENCY_HEDGE,PORT_CASH_EQUIVALENTS,EAI_PORT_AVAILABLE_CASH,VL_DELAY_VALUE,PORT_RATING_MOODYS_SHORT_TERM,PORT_PERCENT_EQUIVALENT,PORT_RATING_SP_SHORT_TERM,PORT_FIRST_TRANSACTION,PORT_RATING_MOODYS,PORT_RATING_SP,PORT_PERCENT_

Récupération de la liste d'actifs en base :

[On prend juste ces valeurs comme dans l'exemple ou toutes les valeurs des assets ?]

In [33]:
def get_df_asset(asset_date, df_currency_rate, asset_id='ASSET_DATABASE_ID', asset_label='LABEL',
                 asset_type='TYPE', asset_last_close_value='LAST_CLOSE_VALUE_IN_CURR'):
    data = get_data('asset?columns={}&columns={}&columns={}&columns={}&date={}'
                   .format(asset_id, asset_label, asset_type, asset_last_close_value, asset_date))
    df_asset = pd.read_json(data)
    df_asset = df_convert_type(df_asset, df_currency_rate)
    return df_asset

df_ETF = get_df_asset('2016-06-01', df_currency_rate)
df_ETF

,ASSET_DATABASE_ID,LAST_CLOSE_VALUE_IN_CURR,TYPE,LABEL
0,1845,36.851000000000,STOCK,1&1 DRILLISCH AG
1,1846,40.900000000000,STOCK,1000MERCIS
2,2122,39.370000000000,FUND,1818-VEGA EUR CO
3,2123,1670.650000000000,FUND,29 HAUSSMANN CROISSANCE EUROPE D
4,2124,NaN,FUND,29 HAUSSMANN EQUILIBRE
...,...,...,...,...
475,1879,3.780000000000,STOCK,YAMANA GOLD INC
476,1997,9.460000000000,STOCK,YMAGIS SA
477,1991,1.000000000000,STOCK,ZCCM INVESTMEN - B
478,2106,40.730337078639,STOCK,ZOETIS INC


Différence entre LAST_CLOSE_VALUE_IN_CURR et LAST_CLOSE_VALUE ??

Récupération des valeurs de cotation:

Cotation de l'actif 1845 en fonction de l'id et de la date de debut et fin :

In [34]:
def get_cotation_values(asset_id, start_date, end_date, df_currency_rate):
    data = get_data('asset/{}/quote?start_date={}&end_date={}'
                    .format(asset_id, start_date, end_date))
    df_cotation = pd.read_json(data)
    df_cotation = df_convert_type(df_cotation, df_currency_rate)
    return df_cotation

df_cotation = get_cotation_values('1845', '2016-06-01', '2020-09-30', df_currency_rate)
df_cotation
    

,date,nav,gross,volume,high,real_close_price,feed_source,low,ask,asset,pl,bid,close,return,open
0,2016-06-01,36.851000000000,36.851000000000,143199.0,37.125,36.851000000000,-10.0,36.675,36.799,1845,0.037000000000,36.723,36.851000000000,0.001005052426,36.745
1,2016-06-02,36.900000000000,36.900000000000,147480.0,37.145,36.900000000000,-10.0,36.594,36.999,1845,0.049000000000,36.946,36.900000000000,0.001329678978,36.786
2,2016-06-03,36.052000000000,36.052000000000,190934.0,37.200,36.052000000000,-10.0,35.800,36.277,1845,-0.848000000000,36.219,36.052000000000,-0.022981029810,36.900
3,2016-06-06,36.540000000000,36.540000000000,82279.0,36.775,36.540000000000,-10.0,36.082,36.587,1845,0.488000000000,36.534,36.540000000000,0.013536003550,36.190
4,2016-06-07,37.246000000000,37.246000000000,119105.0,37.347,37.246000000000,-10.0,36.470,37.172,1845,0.706000000000,37.150,37.246000000000,0.019321291735,36.590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,2020-09-24,27.091248954261,27.091248954261,NaN,NaN,27.091248954261,-10.0,NaN,NaN,1845,-0.550081510430,NaN,27.091248954261,-0.019900688613,NaN
1117,2020-09-25,26.877384954417,26.877384954417,NaN,NaN,26.877384954417,-10.0,NaN,NaN,1845,-0.213863999844,NaN,26.877384954417,-0.007894209684,NaN
1118,2020-09-28,26.634573242213,26.634573242213,NaN,NaN,26.634573242213,-10.0,NaN,NaN,1845,-0.242811712203,NaN,26.634573242213,-0.009034052703,NaN
1119,2020-09-29,26.368478372669,26.368478372669,NaN,NaN,26.368478372669,-10.0,NaN,NaN,1845,-0.266094869544,NaN,26.368478372669,-0.009990581307,NaN


Clean le dataframe df_ETF pour garder seulement les actif qui représentent un %NAV du portefeuille entre 1 et 10% à la date du 01/06/2016 :

1 <= nav <= 10
ou
1 < nav < 10 ????


et si l'asset n'a pas de nav à la date du 01/06/2016, on fait quoi ? on prend la valeur du jour d'avant ?

In [35]:
def keep_only_asset_with_suitable_nav(df_asset, date, nav_min, nav_sup, df_currency_rate):
    assets_to_keep = []
    for index, row in df_asset.iterrows():
        asset_id = row['ASSET_DATABASE_ID']
        df_cot = get_cotation_values(asset_id, date, date, df_currency_rate)
        if df_cot.empty:
            continue
        nav = df_cot.loc[df_cot['asset'] == asset_id]['nav'][0]
        if 1 <= nav and nav <= 10:
            assets_to_keep.append(asset_id)
    df_asset = df_asset.loc[df_asset['ASSET_DATABASE_ID'].isin(assets_to_keep)]
    return df_asset

df_ETF = keep_only_asset_with_suitable_nav(df_ETF, '2016-06-01', 1, 10, df_currency_rate)
df_ETF

,ASSET_DATABASE_ID,LAST_CLOSE_VALUE_IN_CURR,TYPE,LABEL
8,1849,6.730,STOCK,ABC ARBITRAGE
22,1851,1.200,STOCK,ACTEOS
23,1852,5.590,STOCK,ACTIA GROUP
30,1856,4.543,STOCK,AEGON NV
31,1857,7.800,STOCK,AFONE
...,...,...,...,...
461,1781,9.188,STOCK,TELEFONICA
472,1940,8.900,STOCK,VOLTALIA
475,1879,3.780,STOCK,YAMANA GOLD INC
476,1997,9.460,STOCK,YMAGIS SA


Il ne reste plus que seulement 71 actifs ??

Récupération de la liste des ratios :

In [36]:
def get_ratio(df_currency_rate):
    data = get_data('ratio')
    df_ratio = pd.read_json(data)
    #df_ratio = df_convert_type(df_ratio, df_currency_rate)
    return df_ratio

df_ratio = get_ratio(df_currency_rate)
df_ratio

,id,type,name,is_benchmark_needed,is_percent
0,11,Ratio,Correlation,True,False
1,7,Ratio,Exposition action,False,True
2,15,Ratio,Exposition action,False,True
3,8,Ratio,RDT,False,True
4,13,Ratio,Rendement,False,True
5,9,Ratio,Rendement_annualise,False,True
6,12,Ratio,Sharpe,False,False
7,14,Ratio,VaR historique,False,True
8,10,Ratio,Volatilite,False,True


[FIX portfolio ID] Récupération de la composition d’un portefeuille :

In [37]:
def get_portfolio(portfolio_id):
    data = get_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id))
    df_portfolio = pd.read_json(data)
    df_portfolio = df_convert_type(df_portfolio, df_currency_rate)
    return df_portfolio
    #keep seulement list assets ?

portfolio = get_portfolio(564)
portfolio

ValueError: If using all scalar values, you must pass an index

[FIX portfolio ID] MAJ de la composition d’un portefeuille :


In [38]:
def portfolio_json_assets(assets):
    json = ''
    for asset in assets:
        json += '{{"asset":{{"asset": {}, "quantity": {}}}}},'.format(asset["asset"], asset["quantity"])
    return json[:-1]
    
def seralize_portfolio_content(df_portfoliot, assets):
    label = "EPITA_PTF_1"
    currency = "EUR"
    date = "2013-06-14"
    assets = portfolio_json_assets(assets) 
    res = '[{{"label": "{}", "currency": {{"code": "{}"}}, "type": "front", "values": {{"{}": [{}]}}}}]'.format(label, currency, date, assets)
    return res

def deseralize_portfolio_content(json_portfolio):
    json_portfolio = pd.json_normalize(json_portfolio)
    
    cols = [col for col in json_portfolio.columns if col.split('.')[0] == 'values']
    json_portfolio = json_portfolio[cols]
    json_portfolio.columns = json_portfolio.columns.map(lambda x: x.split(".")[0])

    json_assets = json_portfolio['values'][0]
    assets = []
    for asset in json_assets:
        assets.append(asset['asset'])
    return assets

json_portfolio = json.loads(seralize_portfolio_content('', [{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]))
print(json_portfolio)

assets = deseralize_portfolio_content(json_portfolio)
assets

[{'label': 'EPITA_PTF_1', 'currency': {'code': 'EUR'}, 'type': 'front', 'values': {'2013-06-14': [{'asset': {'asset': 42, 'quantity': 2}}, {'asset': {'asset': 1, 'quantity': 1}}]}}]


[{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]

In [39]:
def put_portfolio(portfolio_id, assets):
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), assets)


assets = [{'asset': 42, 'quantity': 2}, {'asset': 1, 'quantity': 1}]
content = seralize_portfolio_content('', assets)

put_portfolio(564, content)

[TODO] Calcul de ratios :

In [ ]:
#Call API to check

[TODO] Functions :

In [40]:
def volatility(returns, nb_period_per_year): #X : col du df
    return returns.std() * np.sqrt(nb_period_per_year)

volatility(df_cotation['close'].values, 365)

210.77009385378707

In [41]:
def beta(X, Y):
    return np.cov(X, Y) / np.var(Y)

In [42]:
def correlation(Ra, Rb):
    return np.cov(Ra, Rb) / np.sqrt(np.var(Ra) * np.var(Rb))

Chaque actif doit représenter un %NAV du portefeuille entre 1 et 10% à la date du 01/06/2016 :

In [43]:
def rendement_Nav(Nav, Nav_prev):
    return (Nav - Nav_prev) / Nav_prev
    #R - R.shift(1))/R.shift(1)

In [ ]:
def rendement_annual():
    

On évaluera le sharpe sur la période du 01/06/2016 au 30/09/2020:

In [44]:
def return_rate(X_end, X_begin, nb_days):
    return (1 + (X_end - X_begin) / X_begin)**(365/nb_days) - 1

In [45]:
def sharpe_ratio(return_rate, nb_period_per_year, Rf=0.05):
    volatility = volatility(returns, nb_period_per_year)
    return (return_rate - Rf) / volatility

In [ ]:
def value_at_risk(Ra, Threshold):
    

In [ ]:
def action_exposure():

Contraites :

50% d'actions  (en montant pas en quantité)

Le portefeuille doit être exactement composé d’un minimum de 15 actifs, et d’un maximum de 40 actifs.

Attention aux devises :

Vérifier bien la devise dans laquelle est exprimée l’actif.

Avez-vous utilisé le bon taux de conversion pour la devise ?

Avez-vous utilisé le taux dans le bon sens ?

Attention aux quotités:

Votre portefeuille comporte-il bien un nombre entier d’actif ? Ou avez-vous acheté 0.0001 action Total ?

L’actif est-il achetable par unité, dizaine, centaines ? le prix donné est-il un prix par unité, dizaine, centaine ?

Le prix de l’actif est-il exprimé en unité ou en centimes ?

Si vous avez arrondis à postériori les quotités, respectez-vous bien toujours les normes fixées pour le portefeuille ?